<a href="https://colab.research.google.com/github/adhello007/PersonalProjects-DeepLearning/blob/main/ObjectDetectionV2_YoloV5ResnetBacked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
import os
import numpy as np
import pandas as pd
os.listdir('/root')

['.bashrc',
 '.profile',
 '.jupyter',
 '.local',
 '.tmux.conf',
 '.cache',
 '.ipython',
 '.config',
 '.npm',
 '.wget-hsts',
 '.keras',
 '.launchpadlib']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle #xreate a directory in the root folder

In [ ]:
!cp '/content/drive/MyDrive/kaggle.json' ~/.kaggle/ #copy the json file from drive to root folder for the api to work

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json #Giving permissioni to read and write the json file

In [ ]:
!kaggle datasets download andrewmvd/road-sign-detection

 98% 214M/218M [00:01<00:00, 196MB/s]
100% 218M/218M [00:01<00:00, 169MB/s]


In [ ]:
!unzip road-sign-detection.zip

In [ ]:
import cv2
import xml.etree.ElementTree as xt
from bs4 import BeautifulSoup
from glob import glob

In [ ]:
#Now lets clone Yolov5 Repo
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15996, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 15996 (delta 100), reused 131 (delta 85), pack-reused 15831
Receiving objects: 100% (15996/15996), 14.58 MiB | 21.30 MiB/s, done.
Resolving deltas: 100% (10974/10974), done.


In [ ]:
yolov5 = '/content/yolov5'
os.chdir(yolov5) #cd = change dir. so current directory is yolov5 dir


In [ ]:
os.getcwd()

'/content/yolov5'

In [ ]:
xml_eg = '/content/annotations/road0.xml'

with open(xml_eg,'r') as f:
  data = f.read()
  content = BeautifulSoup(data,'xml')

In [ ]:
content

<?xml version="1.0" encoding="utf-8"?>
<annotation>
<folder>images</folder>
<filename>road0.png</filename>
<size>
<width>267</width>
<height>400</height>
<depth>3</depth>
</size>
<segmented>0</segmented>
<object>
<name>trafficlight</name>
<pose>Unspecified</pose>
<truncated>0</truncated>
<occluded>0</occluded>
<difficult>0</difficult>
<bndbox>
<xmin>98</xmin>
<ymin>62</ymin>
<xmax>208</xmax>
<ymax>232</ymax>
</bndbox>
</object>
</annotation>

In [ ]:
annotations_path = '/content/annotations'
output_path = '/content/yolov5'

In [ ]:
#Took all the values into a list

img_name_list = []
width_list = []
height_list = []
label_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []

path = sorted(glob('/content/annotations/*.xml'))

for file in path:
  info = xt.parse(file)
  root = info.getroot()

  inl = root.find('filename').text
  img_name_list.append(inl)


  size = root.find('size')
  wl = size.find('width').text
  width_list.append(wl)


  hl = size.find('height').text
  height_list.append(hl)

  obj = root.find('object')
  label = obj.find('name').text
  label_list.append(label)

  bndbox = obj.find('bndbox')
  xmin = (bndbox.find('xmin').text)
  xmax = (bndbox.find('xmax').text)
  ymin = (bndbox.find('ymin').text)
  ymax = (bndbox.find('ymax').text)


  xmin_list.append(xmin)
  ymin_list.append(ymin)
  xmax_list.append(xmax)
  ymax_list.append(ymax)


In [ ]:
print(img_name_list[:3],
width_list[:3],
height_list[:3],
label_list[:3],
xmin_list[:3],
ymin_list[:3],
xmax_list[:3],
ymax_list[:3])

['road0.png', 'road1.png', 'road10.png'] ['267', '400', '400'] ['400', '283', '267'] ['trafficlight', 'trafficlight', 'trafficlight'] ['98', '154', '106'] ['62', '63', '3'] ['208', '258', '244'] ['232', '281', '263']


Convert this into a pd dataframe

In [ ]:
df = pd.DataFrame({
    'img_name' : img_name_list,
    'width' : width_list,
    'height' : height_list,
    'xmin' : xmin_list,
    'ymin' : ymin_list,
    'xmax' : xmax_list,
    'ymax' : ymax_list,
    'label' : label_list})

df.head()

,img_name,width,height,xmin,ymin,xmax,ymax,label
0,road0.png,267,400,98,62,208,232,trafficlight
1,road1.png,400,283,154,63,258,281,trafficlight
2,road10.png,400,267,106,3,244,263,trafficlight
3,road100.png,400,385,35,5,363,326,speedlimit
4,road101.png,400,200,195,7,392,194,speedlimit


In [ ]:
!pip install pybboxes #help convert data to yolo format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pybboxes as pbx
from collections import defaultdict
from tqdm import tqdm
import shutil

In [ ]:
classes = df['label'].unique().tolist()

In [ ]:
classes

['trafficlight', 'speedlimit', 'crosswalk', 'stop']

In [ ]:
#Add class number associated to classes
df['class'] = df['label'].apply(lambda x: classes.index(x))

In [ ]:
df.head()

,img_name,width,height,xmin,ymin,xmax,ymax,label,class
0,road0.png,267,400,98,62,208,232,trafficlight,0
1,road1.png,400,283,154,63,258,281,trafficlight,0
2,road10.png,400,267,106,3,244,263,trafficlight,0
3,road100.png,400,385,35,5,363,326,speedlimit,1
4,road101.png,400,200,195,7,392,194,speedlimit,1


In [ ]:
img_dict = defaultdict(list) #created an empty list, we'll fill it now

for idx in tqdm(range(len(df))): #just a simple for loop
  sample_label_list = []
  img_name = df.loc[idx,'img_name'] #accessing idx'th value of the img_name col
  xmin = df.loc[idx,'xmin']
  ymin = df.loc[idx,'ymin']
  xmax= df.loc[idx,'xmax']
  ymax = df.loc[idx,'ymax']
  class_num = df.loc[idx,'class']
  W, H = int(df.loc[idx, 'width']), int(df.loc[idx, 'height'])


  #visual object classes
  voc_bbox = (int(xmin),int(ymin),int(xmax),int(ymax))

  x_center, y_center, w, h = pbx.convert_bbox(voc_bbox, from_type="voc", to_type="yolo", image_size=(W,H))
   #The function takes the bounding box coordinates in the "voc" format and converts them to the "yolo" format.
  sample_label_list.append(str(class_num))
  sample_label_list.append(str(x_center))
  sample_label_list.append(str(y_center))
  sample_label_list.append(str(w))
  sample_label_list.append(str(h))
  line = ' '.join(sample_label_list)
  #this line above converts eg: ['1', '0.5', '0.7', '0.2', '0.3'] to 1 0.5 0.7 0.2 0.3 format

  img_dict[img_name].append(line)

100%|██████████| 877/877 [00:00<00:00, 6313.19it/s]


the keys are image names ('image1.jpg', 'image2.jpg', 'image3.jpg'), and the corresponding values are lists of bounding box labels. Each bounding box label is a string that represents the class number and the bounding box coordinates in the YOLO format.

In [ ]:
img_dict

defaultdict(list,
            {'road0.png': ['0 0.5730337078651685 0.3675 0.41198501872659177 0.425'],
             'road1.png': ['0 0.515 0.607773851590106 0.26 0.7703180212014135'],
             'road10.png': ['0 0.4375 0.49812734082397003 0.345 0.9737827715355806'],
             'road100.png': ['1 0.4975 0.42987012987012985 0.82 0.8337662337662337'],
             'road101.png': ['1 0.73375 0.5025 0.4925 0.935'],
             'road102.png': ['1 0.35 0.5299625468164794 0.515 0.7752808988764045'],
             'road103.png': ['1 0.4825 0.49335548172757476 0.52 0.8205980066445183'],
             'road104.png': ['1 0.49 0.49313186813186816 0.74 0.9093406593406593'],
             'road105.png': ['1 0.5133333333333333 0.65375 0.8466666666666667 0.5775'],
             'road106.png': ['1 0.4943820224719101 0.47 0.23220973782771537 0.19'],
             'road107.png': ['1 0.5046875 0.33625 0.446875 0.4875'],
             'road108.png': ['1 0.45 0.5426829268292683 0.03 0.044715447154471545'],
 

In [ ]:
img_dict['road0.png']

['0 0.5730337078651685 0.3675 0.41198501872659177 0.425']

In [ ]:
#lets make a labels dir in data folder of yolov5
labels_dir = f'{output_path}/data/labels'
if os.path.exists(labels_dir):
  shutil.rmtree(labels_dir) #removes the directory and all its contents
os.mkdir(labels_dir)

By executing this code, you will generate a .txt file for each image in img_dict with the corresponding bounding box labels. The file names will match the image names (without the file extension), and each file will contain the lines representing the bounding box labels associated with that image.

In [ ]:
#We will Generate a .txt file for each image
for img_name, lines in img_dict.items(): #keys and values
  img_name = img_name.split('.')[0] #name of the image 'road0'
  with open(f'{labels_dir}/{img_name}.txt','w') as f:
    for line in lines:
      f.write(line)
      f.write('\n')

Generate Train Val Split

In [ ]:
import os
import shutil
from random import shuffle



In [ ]:
input_path = f'{yolov5}/data'


In [ ]:

images_path = '/content/images'
labels_path = labels_dir




In [ ]:
train_dir = output_path + '/data/train'
val_dir = output_path + '/data/val'

if os.path.exists(train_dir):
  shutil.rmtree(train_dir) #it will delete its content and the folder

if os.path.exists(val_dir):
  shutil.rmtree(val_dir) #it will delete its content and the folder


os.mkdir(train_dir)
os.mkdir(val_dir)

# train, val each containing images and labels folders
os.mkdir(train_dir + '/images')
os.mkdir(train_dir + '/labels')

os.mkdir(val_dir + '/images')
os.mkdir(val_dir + '/labels')



In [ ]:
#shuffle the image file names before splitting into train and val data
files = os.listdir(images_path)
shuffle(files)


split(files, ratio): Splits a list of files into two sublists based on a given ratio.
copy_files(images_path, labels_path, destination_path, files): Copies image and label files to a destination path.

In [ ]:
def split(files, ratio):
    elements = len(files)
    middle = int(elements * ratio)
    return [files[:middle], files[middle:]]

def copy_files(images_path, labels_path, destination_path, files):

    for file_name in files:
        file_name = file_name.split('.')[0]

        src = images_path + f'/{file_name}.png'
        dst = destination_path + '/images'
        shutil.copy(src, dst)

        src = labels_path + f'/{file_name}.txt'
        dst = destination_path + '/labels'
        shutil.copy(src, dst)

In [ ]:
print(images_path,
labels_path,
train_dir)

/content/images /content/yolov5/data/labels /content/yolov5/data/train


In [ ]:
# Split and copy files in train and val folder
train_ratio = 0.75
train_files, val_files = split(files, train_ratio)

copy_files(images_path, labels_path, train_dir, train_files)
copy_files(images_path, labels_path, val_dir, val_files)

In [ ]:
assert (len(os.listdir(train_dir + '/images')) + len(os.listdir(val_dir + '/images')) == len(os.listdir(images_path)))

Generate .yaml yolo file


In [ ]:
pip install ultralytics


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 16.3 MB/s eta 0:00:00


In [ ]:
with open(f'{output_path}/data/sign_data.yaml','w') as f:
  f.write('train: /content/yolov5/data/train/images\n')
  f.write('val:  /content/yolov5/data/val/images\n')
  f.write('nc: 4\n')
  f.write(f"names: {classes}")


Time to train the model!


In [ ]:
epochs = 20
!python train.py --img 640 --batch 16 --epochs {epochs} --data sign_data.yaml --weights yolov5s.pt

requirements: Ultralytics requirement "gitpython" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 223.6 MB/s eta 0:00:00

requirements: 1 package updated per ['gitpython']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

train: weights=yolov5s.pt, cfg=, data=sign_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=No